# Lab Assignment Seven: Sequential Network Architectures
In this lab, you will select a prediction task to perform on your dataset, evaluate a sequential architecture and tune hyper-parameters. If any part of the assignment is not clear, ask the instructor to clarify.

This report is worth 10% of the final grade. Please upload a report (one per team) with all code used, visualizations, and text in a rendered Jupyter notebook. Any visualizations that cannot be embedded in the notebook, please provide screenshots of the output. The results should be reproducible using your report. Please carefully describe every assumption and every step in your report.

## Dataset Selection

Select a dataset that is text. That is, the dataset should be text data. In terms of generalization performance, it is helpful to have a medium sized dataset of similar sized text documents. It is fine to perform binary classification or multi-class classification. The classification should be "many-to-one" sequence classification.



## Preparation (3 points total)
[1 points] Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed). Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created). Discuss methods of tokenization in your dataset as well as any decisions to force a specific length of sequence.  
[1 points] Choose and explain what metric(s) you will use to evaluate your algorithm’s performance. You should give a detailed argument for why this (these) metric(s) are appropriate on your data. That is, why is the metric appropriate for the task (e.g., in terms of the business case for the task). Please note: rarely is accuracy the best evaluation metric to use. Think deeply about an appropriate measure of performance.
[1 points] Choose the method you will use for dividing your data into training and testing (i.e., are you using Stratified 10-fold cross validation? Shuffle splits? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. Convince me that your train/test splitting method is a realistic mirroring of how an algorithm would be used in practice.

## Modeling (6 points total)
[3 points] Investigate at least two different sequential network architectures (e.g., a CNN and a Transformer). Alternatively, you may also choose a recurrent network and Transformer network. Be sure to use an embedding layer (try to use a pre-trained embedding, if possible). Adjust one hyper-parameter of each network to potentially improve generalization performance (train a total of at least four models). Visualize the performance of training and validation sets versus the training iterations, showing that the models converged.
[1 points] Using the best parameters and architecture from the Transformer in the previous step, add a second Multi-headed self attention layer to your network. That is, the input to the second attention layer should be the output sequence of the first attention layer.  Visualize the performance of training and validation sets versus the training iterations.
[2 points] Use the method of train/test splitting and evaluation criteria that you argued for at the beginning of the lab. Visualize the results of all the models you trained.  Use proper statistical comparison techniques to determine which method(s) is (are) superior.  

## Exceptional Work (1 points total)
You have free reign to provide additional analyses.
One idea (required for 7000 level students to do one of these options):
Use the pre-trained ConceptNet Numberbatch embedding and compare to pre-trained GloVe. Which method is better for your specific application?

In [ ]:
!pip install tensorflow_addons


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 6.3 MB/s eta 0:00:00


In [18]:
import subprocess
import platform
import os
import itertools
import time
from IPython.display import display, HTML, Markdown, clear_output

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def clear_screen():
    time.sleep(2)
    print("Clearing screen...")
    time.sleep(2)
    clear_output()

clear_screen()

In [15]:
import pandas as pd
import zipfile
import os
from google.colab import drive
from google.colab import files

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Dataset Selection

https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset


This dataset contains 5,574 English SMS messages labeled as spam or not spam (ham). The dataset contains one message per line. Each line is composed of two columns the label and the textual data. The label in this data set is a binary classifier with results of either ham or spam. Therefore representing a many-to-one relationship. This dataset originally labeled the columns v1 for the label and v2 for the raw text data. To provide readability to our code we changed the column names to be labeled label and text respectively.


Acknowledgments
The original dataset can be found here.

Reference to previous paper: Almeida, T.A., GÃ³mez Hidalgo, J.M., Yamakami, A. Contributions to the Study of SMS Spam Filtering: New Collection and Results. Proceedings of the 2011 ACM Symposium on Document Engineering (DOCENG'11), Mountain View, CA, USA, 2011.
Reference to web page: http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/


In [27]:
# Read in our dataset
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Lab7_ML/spam 2.csv", encoding='latin1')

# Renaming the columns for readability purposes
df.rename(columns={'v1': 'Label'}, inplace=True)
df.rename(columns={'v2': 'Text'}, inplace=True)

df.head()


,Label,Text,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('Label').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('Text').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Text'].value_counts()
    for x_label, grp in _df_2.groupby('Label')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Label')
_ = plt.ylabel('Text')

## Evaluation Method



For evaluating our model's performance, we will be using recall. Between the two types of classifications spam and ham it is important to avoid marking possibly important messages as spam/junk mail. Misclassification of text messages could lead to dissatisfaction and the loss of important information. For example, a missing credit card statement, an appointment reminder, a package delivery, and much more. Therefore our models will be evaluated on their ability to positively classify these messages as spam or ham.

Beyond recall, we use metrics such as accuracy, misclassificaiton, precision, recall, and F1-score to measure and compare algorithm performance.

Accuracy is a measure of the overall correctness of a model's predictions, calculated as the ratio of the number of correct predictions to the total number of predictions.

Misclass, or misclassification, refers to the instances where a model's prediction does not match the actual class label of a data point.

Precision is a measure of how well a model correctly identifies positive instances among the predicted positive instances.

Recall, also known as sensitivity or the true positive rate, is a measure of how well a model identifies all the positive instances among the actual positive instances.

F1-score is the "harmonic mean" of precision and recall, providing a single value that balances the trade-off between precision and recall.


Additionally, we will evaluate the performance of a sequential network based on its training time and computational resources required. It is important to take into consideration the amount of time and resources our models will need because we want to avoid crashing the messaging device while predicting if a message is spam or ham.  


We want to consider all of these metrics since not only do we want our models to perform well, but we want to achieve the highest true positive rate for both classes as possible.

## Dataset Splitting

## Data Preparation

The first step in preprocessing our data for sequential networks is to split the data into training and testing sets.

In [28]:
X_train, X_test, y_train, y_test = train_test_split(df['Label'], df['Text'], test_size=0.25)



**The second step is to tokenize the data. **
Tokenizing the data is breaking the text into a sequence of tokens. These tokens are often words or charecters in the dataset. It is important to tokenize a dataset to reduce the dimensionality of the dataset making the network easier to learn! Tokenizing the dataset can also emphasize the important parts of the text which can therefore improve the accuracy of the network's prediction. Lastly, tokenizing the dataset can help improve the stability of a dataset byt making the dataset more consistent.

In the cell below you can see we set num_words to 10,000 this ensures that we will include only the first 10,000 frequent words in our text vectorization later.


In [22]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

**The third step is to force a specific length on our dataset. **
To force a specific length of sequence on your dataset you can use the pad_sequences() function from the keras preprocessing library. This function takes a max length argument and either truncates your sequences or adds zeros to force a specific length on the dataset.

In the cell below you can see we pass the split dataset through the pad_sequences() function to ensure each sequence will be at a max length of 100.





In [29]:
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=100)
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=100)

y_train = pad_sequences(tokenizer.texts_to_sequences(y_train), maxlen=100)
y_test = pad_sequences(tokenizer.texts_to_sequences(y_test), maxlen=100)

**The last step is to one-hot encode the labels. **

Since our labels are categorical values of spam or not spam (also known as ham) it is important to convert these values to binary values. The label will take on a value of 1 if the vector is spam and a 0 if it is ham.

It is important to one-hot encode categorical values in our dataset to ensure our model can learn the relationship between the different categories.


In [30]:
from keras.utils import to_categorical

y_train = np.array(y_train)
y_test = np.array(y_test)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)